# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [303]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

Nhl_df=pd.read_csv("data/nhl.csv")
Cities=pd.read_html("data/wikipedia_data.htm")[1]
Cities=Cities.iloc[:-1,[0,3,5,6,7,8]]

columnas = ["team","W","L","year"]
nhl_df = Nhl_df[columnas].copy()
nhl_df = nhl_df[nhl_df['year'] == 2018].copy()
nhl_df['Team'] = nhl_df['team'].apply(lambda x: x.split()[-1])  # Obtén la última palabra como el equipo
nhl_df['City'] = nhl_df['team'].apply(lambda x: ' '.join(x.split()[:-1]))  # Obtén todas las palabras excepto la última como el nombre de la ciudad
nhl_df = nhl_df[['City', 'Team', 'W', 'L', 'year']]
nhl_df['Team'] = nhl_df['Team'].str.rstrip('*')
nhl_df = nhl_df.drop([0, 9, 18, 26]).reset_index(drop=True)
nhl_df.at[12, 'City'] = 'New York'
replacement_dict = {
    'New York': 'New York City',
    'Anaheim': 'Los Angeles',
    'San Jose': 'San Francisco Bay Area',
    'Dallas': 'Dallas–Fort Worth',
    'Florida':'Miami–Fort Lauderdale',
    'Tampa Bay': 'Tampa Bay Area',
    'New Jersey': 'New York City',
    'Arizona': 'Phoenix',
    'Minnesota': 'Minneapolis–Saint Paul',
    'Colorado': 'Denver',
    'Carolina': 'Raleigh',
    'Vegas Golden': 'Las Vegas',
    'Toronto Maple': 'Toronto',
    'Washington': 'Washington, D.C.',
    'Columbus Blue': 'Columbus',
    'Detroit Red': 'Detroit',
    
}
nhl_df['City'] = nhl_df['City'].replace(replacement_dict)
#nhl_df = nhl_df.groupby('City')['Team'].apply(lambda x: ' '.join(sorted(x, reverse=True))).reset_index()
# Convertir las columnas 'W' y 'L' a tipos numéricos
nhl_df['W'] = pd.to_numeric(nhl_df['W'])
nhl_df['L'] = pd.to_numeric(nhl_df['L'])

nhl_df

,City,Team,W,L,year
0,Tampa Bay Area,Lightning,54,23,2018
1,Boston,Bruins,50,20,2018
2,Toronto,Leafs,49,26,2018
3,Miami–Fort Lauderdale,Panthers,44,30,2018
4,Detroit,Wings,30,39,2018
5,Montreal,Canadiens,29,40,2018
6,Ottawa,Senators,28,43,2018
7,Buffalo,Sabres,25,45,2018
8,"Washington, D.C.",Capitals,49,26,2018
9,Pittsburgh,Penguins,47,29,2018


In [304]:
columnas = ["Metropolitan area","Population (2016 est.)[8]","NHL"]
cities = Cities[columnas].copy()

cities['NHL'] = cities['NHL'].str.extract('([a-zA-Z\s]+)')
cities = cities.dropna(subset=['NHL'])
cities = cities[~cities['NHL'].str.contains('note')].reset_index(drop=True)
cities


,Metropolitan area,Population (2016 est.)[8],NHL
0,New York City,20153634,Rangers Islanders Devils
1,Los Angeles,13310447,Kings Ducks
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars
5,"Washington, D.C.",6131977,Capitals
6,Philadelphia,6070500,Flyers
7,Boston,4794447,Bruins
8,Minneapolis–Saint Paul,3551036,Wild
9,Denver,2853077,Avalanche


In [305]:
merged_df = pd.merge(nhl_df, cities, left_on='City', right_on='Metropolitan area', how='inner')
merged_df = merged_df.drop(['Metropolitan area','NHL','year'], axis=1)

# Agrupar por 'City' y 'Team'
grouped_df = merged_df.groupby(['City', 'Team','Population (2016 est.)[8]']).agg({
    'W': 'sum',
    'L': 'sum'
}).reset_index()

# Calcular las tasas de victorias y derrotas para cada equipo en cada ciudad
grouped_df['Win_Loss_Ratio'] = grouped_df['W'] / (grouped_df['W'] + grouped_df['L'])

# Agrupar por 'City' y calcular la media de las tasas de victorias y derrotas para cada equipo
merged_df = grouped_df.groupby('City').agg({
    #'Team': lambda x: ' '.join(x),
    'Win_Loss_Ratio': 'mean',
    'Population (2016 est.)[8]': lambda x: x.iloc[0]
}).reset_index()
merged_df['Population (2016 est.)[8]'] = pd.to_numeric(merged_df['Population (2016 est.)[8]'], errors='coerce')

merged_df



,City,Win_Loss_Ratio,Population (2016 est.)[8]
0,Boston,0.714286,4794447
1,Buffalo,0.357143,1132804
2,Calgary,0.513889,1392609
3,Chicago,0.458333,9512999
4,Columbus,0.600000,2041520
5,Dallas–Fort Worth,0.567568,7233323
6,Denver,0.589041,2853077
7,Detroit,0.434783,4297617
8,Edmonton,0.473684,1321426
9,Las Vegas,0.680000,2155664


In [306]:
population_by_region = merged_df['Population (2016 est.)[8]'].tolist()

win_loss_by_region = merged_df['Win_Loss_Ratio'].tolist()

pearson =stats.pearsonr(population_by_region, win_loss_by_region)
pearson[0]


0.012486162921209895

In [307]:
def nhl_correlation(): 
    population_by_region = merged_df['Population (2016 est.)[8]'].tolist()
    
    win_loss_by_region = merged_df['Win_Loss_Ratio'].tolist()

    raise NotImplementedError()

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [308]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

Nba_df=pd.read_csv("data/nba.csv")
cities=pd.read_html("data/wikipedia_data.htm")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

columnas = ["team","W","L","year"]
nba_df = Nba_df[columnas].copy()
nba_df = nba_df[nba_df['year'] == 2018].copy()

# Crear las columnas 'Team' y 'City'
nba_df['Team'] = nba_df['team'].apply(lambda x: re.sub(r'\* \(.*\)', '', x).strip())
nba_df['City'] = nba_df['team'].apply(lambda x: re.sub(r'\* \(.*\)', '', x).rsplit(' ', 1)[0])

# Seleccionar columnas relevantes y eliminar filas innecesarias
nba_df = nba_df[['City', 'Team', 'W', 'L', 'year']]
nba_df['Team'] = nba_df['Team'].str.rstrip('*')
#nba_df = nba_df.drop([8, 9]).reset_index(drop=True)  # Ajusta según las filas que deseas eliminar

# Reemplazar nombres de ciudades según el diccionario de reemplazo
replacement_dict_nba = {
    'New York': 'New York City',
    'Anaheim': 'Los Angeles',
    'San Jose': 'San Francisco Bay Area',
    'Dallas': 'Dallas–Fort Worth',
    'Florida':'Miami–Fort Lauderdale',
    'Tampa Bay': 'Tampa Bay Area',
    'New Jersey': 'New York City',
    'Arizona': 'Phoenix',
    'Minnesota': 'Minneapolis–Saint Paul',
    'Colorado': 'Denver',
    'Carolina': 'Raleigh',
    'Vegas Golden': 'Las Vegas',
    'Toronto Maple': 'Toronto',
    'Washington': 'Washington, D.C.',
    'Columbus Blue': 'Columbus',
    'Detroit Red': 'Detroit',
    'Indiana': 'Indianapolis',
    'Miami': 'Miami–Fort Lauderdale',
    'Utah': 'Salt Lake City',
    'Golden State': 'San Francisco Bay Area',
    'LA': 'Los Angeles',
    'LA Clippers': 'Los Angeles',
    'LA Lakers': 'Los Angeles',
    'Brooklyn': 'New York City',
    'Charlotte': 'Charlotte',
    'Memphis': 'Memphis',
    'Portland Trail': 'Portland',
    'Detroit': 'Detroit',
    
    
}
nba_df['City'] = nba_df['City'].replace(replacement_dict_nba)

nba_df['W'] = pd.to_numeric(nba_df['W'])
nba_df['L'] = pd.to_numeric(nba_df['L'])

nba_df

,City,Team,W,L,year
0,Toronto,Toronto Raptors* (1),59,23,2018
1,Boston,Boston Celtics* (2),55,27,2018
2,Philadelphia,Philadelphia 76ers* (3),52,30,2018
3,Cleveland,Cleveland Cavaliers* (4),50,32,2018
4,Indianapolis,Indiana Pacers* (5),48,34,2018
5,Miami–Fort Lauderdale,Miami Heat* (6),44,38,2018
6,Milwaukee,Milwaukee Bucks* (7),44,38,2018
7,"Washington, D.C.",Washington Wizards* (8),43,39,2018
8,Detroit,Detroit Pistons (9),39,43,2018
9,Charlotte,Charlotte Hornets (10),36,46,2018


In [309]:
columnas = ["Metropolitan area","Population (2016 est.)[8]","NBA"]
cities = Cities[columnas].copy()

cities['NBA'] = cities['NBA'].str.extract('([a-zA-Z\s]+)')
cities = cities.dropna(subset=['NBA'])
cities = cities[~cities['NBA'].str.contains('note')].reset_index(drop=True)
cities

,Metropolitan area,Population (2016 est.)[8],NBA
0,New York City,20153634,Knicks Nets
1,Los Angeles,13310447,Lakers Clippers
2,San Francisco Bay Area,6657982,Warriors
3,Chicago,9512999,Bulls
4,Dallas–Fort Worth,7233323,Mavericks
5,"Washington, D.C.",6131977,Wizards
6,Philadelphia,6070500,ers
7,Boston,4794447,Celtics
8,Minneapolis–Saint Paul,3551036,Timberwolves
9,Denver,2853077,Nuggets


In [310]:
merged_df = pd.merge(nba_df, cities, left_on='City', right_on='Metropolitan area', how='inner')
merged_df = merged_df.drop(['Metropolitan area','NBA','year'], axis=1)

# Agrupar por 'City' y 'Team'
grouped_df = merged_df.groupby(['City', 'Team','Population (2016 est.)[8]']).agg({
    'W': 'sum',
    'L': 'sum'
}).reset_index()

# Calcular las tasas de victorias y derrotas para cada equipo en cada ciudad
grouped_df['Win_Loss_Ratio'] = grouped_df['W'] / (grouped_df['W'] + grouped_df['L'])

# Agrupar por 'City' y calcular la media de las tasas de victorias y derrotas para cada equipo
merged_df = grouped_df.groupby('City').agg({
    #'Team': lambda x: ' '.join(x),
    'Win_Loss_Ratio': 'mean',
    'Population (2016 est.)[8]': lambda x: x.iloc[0]
}).reset_index()
merged_df['Population (2016 est.)[8]'] = pd.to_numeric(merged_df['Population (2016 est.)[8]'], errors='coerce')

merged_df

,City,Win_Loss_Ratio,Population (2016 est.)[8]
0,Atlanta,0.292683,5789700
1,Boston,0.670732,4794447
2,Charlotte,0.439024,2474314
3,Chicago,0.329268,9512999
4,Cleveland,0.609756,2055612
5,Dallas–Fort Worth,0.292683,7233323
6,Denver,0.560976,2853077
7,Detroit,0.475610,4297617
8,Houston,0.792683,6772470
9,Indianapolis,0.585366,2004230


In [311]:
population_by_region = merged_df['Population (2016 est.)[8]'].tolist()

win_loss_by_region = merged_df['Win_Loss_Ratio'].tolist()

pearson =stats.pearsonr(population_by_region, win_loss_by_region)
pearson

PearsonRResult(statistic=-0.17657160252844611, pvalue=0.3687474160446298)

In [312]:
def nba_correlation():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [313]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

Mlb_df=pd.read_csv("data/mlb.csv")
cities=pd.read_html("data/wikipedia_data.htm")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


In [314]:
columnas = ["team","W","L","year"]
mlb_df = Mlb_df[columnas].copy()
mlb_df = mlb_df[mlb_df['year'] == 2018].copy()

# Crear las columnas 'Team' y 'City'
mlb_df['Team'] = mlb_df['team'].apply(lambda x: re.sub(r'\* \(.*\)', '', x).strip())
mlb_df['City'] = mlb_df['team'].apply(lambda x: re.sub(r'\* \(.*\)', '', x).rsplit(' ', 1)[0])

# Seleccionar columnas relevantes y eliminar filas innecesarias
mlb_df = mlb_df[['City', 'Team', 'W', 'L', 'year']]
mlb_df['Team'] = mlb_df['Team'].str.rstrip('*')
#nba_df = nba_df.drop([8, 9]).reset_index(drop=True)  # Ajusta según las filas que deseas eliminar

# Reemplazar nombres de ciudades según el diccionario de reemplazo
replacement_dict_nba = {
    'New York': 'New York City',
    'Anaheim': 'Los Angeles',
    'San Jose': 'San Francisco Bay Area',
    'Dallas': 'Dallas–Fort Worth',
    'Florida':'Miami–Fort Lauderdale',
    'Tampa Bay': 'Tampa Bay Area',
    'New Jersey': 'New York City',
    'Arizona': 'Phoenix',
    'Minnesota': 'Minneapolis–Saint Paul',
    'Colorado': 'Denver',
    'Carolina': 'Raleigh',
    'Vegas Golden': 'Las Vegas',
    'Toronto Maple': 'Toronto',
    'Washington': 'Washington, D.C.',
    'Columbus Blue': 'Columbus',
    'Detroit Red': 'Detroit',
    'Indiana': 'Indianapolis',
    'Miami': 'Miami–Fort Lauderdale',
    'Utah': 'Salt Lake City',
    'Golden State': 'San Francisco Bay Area',
    'LA': 'Los Angeles',
    'LA Clippers': 'Los Angeles',
    'LA Lakers': 'Los Angeles',
    'Brooklyn': 'New York City',
    'Charlotte': 'Charlotte',
    'Memphis': 'Memphis',
    'Portland Trail': 'Portland',
    'Boston Red': 'Boston',
    'Chicago White': 'Chicago',
    'Toronto Blue': 'Toronto',
    'San Francisco': 'San Francisco Bay Area',
    'Texas': 'Dallas–Fort Worth',
    'Oakland': 'San Francisco Bay Area',
}
mlb_df['City'] = mlb_df['City'].replace(replacement_dict_nba)

mlb_df['W'] = pd.to_numeric(mlb_df['W'])
mlb_df['L'] = pd.to_numeric(mlb_df['L'])

mlb_df



,City,Team,W,L,year
0,Boston,Boston Red Sox,108,54,2018
1,New York City,New York Yankees,100,62,2018
2,Tampa Bay Area,Tampa Bay Rays,90,72,2018
3,Toronto,Toronto Blue Jays,73,89,2018
4,Baltimore,Baltimore Orioles,47,115,2018
5,Cleveland,Cleveland Indians,91,71,2018
6,Minneapolis–Saint Paul,Minnesota Twins,78,84,2018
7,Detroit,Detroit Tigers,64,98,2018
8,Chicago,Chicago White Sox,62,100,2018
9,Kansas City,Kansas City Royals,58,104,2018


In [315]:
# Seleccionar columnas relevantes de cities
columnas_cities = ["Metropolitan area", "Population (2016 est.)[8]", "MLB"]
cities_subset = cities[columnas_cities].copy()

# Limpiar y procesar datos en cities_subset (ajustar según sea necesario)
cities_subset['MLB'] = cities_subset['MLB'].str.extract('([a-zA-Z\s]+)')
cities_subset = cities_subset.dropna(subset=['MLB'])
cities_subset = cities_subset[~cities_subset['MLB'].str.contains('note')].reset_index(drop=True)

cities_subset


,Metropolitan area,Population (2016 est.)[8],MLB
0,New York City,20153634,Yankees Mets
1,Los Angeles,13310447,Dodgers Angels
2,San Francisco Bay Area,6657982,Giants Athletics
3,Chicago,9512999,Cubs White Sox
4,Dallas–Fort Worth,7233323,Rangers
5,"Washington, D.C.",6131977,Nationals
6,Philadelphia,6070500,Phillies
7,Boston,4794447,Red Sox
8,Minneapolis–Saint Paul,3551036,Twins
9,Denver,2853077,Rockies


In [316]:
# Fusionar DataFrames
merged_df = pd.merge(mlb_df, cities_subset, left_on='City', right_on='Metropolitan area', how='inner')
merged_df = merged_df.drop(['Metropolitan area', 'MLB', 'year'], axis=1)

# Agrupar por 'City' y 'Team'
grouped_df = merged_df.groupby(['City', 'Team', 'Population (2016 est.)[8]']).agg({
    'W': 'sum',
    'L': 'sum'
}).reset_index()

# Calcular las tasas de victorias y derrotas para cada equipo en cada ciudad
grouped_df['Win_Loss_Ratio'] = grouped_df['W'] / (grouped_df['W'] + grouped_df['L'])

# Agrupar por 'City' y calcular la media de las tasas de victorias y derrotas para cada equipo
merged_df = grouped_df.groupby('City').agg({
    'Win_Loss_Ratio': 'mean',
    'Population (2016 est.)[8]': lambda x: x.iloc[0]
}).reset_index()
merged_df['Population (2016 est.)[8]'] = pd.to_numeric(merged_df['Population (2016 est.)[8]'], errors='coerce')

merged_df

,City,Win_Loss_Ratio,Population (2016 est.)[8]
0,Atlanta,0.555556,5789700
1,Baltimore,0.290123,2798886
2,Boston,0.666667,4794447
3,Chicago,0.482769,9512999
4,Cincinnati,0.413580,2165139
5,Cleveland,0.561728,2055612
6,Dallas–Fort Worth,0.413580,7233323
7,Denver,0.558282,2853077
8,Detroit,0.395062,4297617
9,Houston,0.635802,6772470


In [317]:
def mlb_correlation(): 
    population_by_region = merged_df['Population (2016 est.)[8]'].tolist()
    win_loss_by_region = merged_df['Win_Loss_Ratio'].tolist()

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [318]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

Nfl_df=pd.read_csv("data/nfl.csv")
cities=pd.read_html("data/wikipedia_data.htm")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

columnas = ["team","W","L","year"]
nfl_df = Nfl_df[columnas].copy()
nfl_df = nfl_df[nfl_df['year'] == 2018].copy()

# Crear las columnas 'Team' y 'City'
nfl_df['Team'] = nfl_df['team'].apply(lambda x: re.sub(r'\* \(.*\)', '', x).strip())
nfl_df['City'] = nfl_df['team'].apply(lambda x: re.sub(r'\* \(.*\)', '', x).rsplit(' ', 1)[0])

# Seleccionar columnas relevantes y eliminar filas innecesarias
nfl_df = nfl_df[['City', 'Team', 'W', 'L', 'year']]
nfl_df['Team'] = nfl_df['Team'].str.rstrip('*')

# Reemplazar nombres de ciudades según el diccionario de reemplazo
replacement_dict_nfl = {
    'New York': 'New York City',
    'Anaheim': 'Los Angeles',
    'San Jose': 'San Francisco Bay Area',
    'Dallas': 'Dallas–Fort Worth',
    'Florida':'Miami–Fort Lauderdale',
    'Tampa Bay': 'Tampa Bay Area',
    'New Jersey': 'New York City',
    'Arizona': 'Phoenix',
    'Minnesota': 'Minneapolis–Saint Paul',
    'Colorado': 'Denver',
    'Vegas Golden': 'Las Vegas',
    'Toronto Maple': 'Toronto',
    'Washington': 'Washington, D.C.',
    'Columbus Blue': 'Columbus',
    'Detroit Red': 'Detroit',
    'Indiana': 'Indianapolis',
    'Miami': 'Miami–Fort Lauderdale',
    'Utah': 'Salt Lake City',
    'Golden State': 'San Francisco Bay Area',
    'LA': 'Los Angeles',
    'LA Clippers': 'Los Angeles',
    'LA Lakers': 'Los Angeles',
    'Brooklyn': 'New York City',
    'Memphis': 'Memphis',
    'Portland Trail': 'Portland',
    'Boston Red': 'Boston',
    'Chicago White': 'Chicago',
    'Toronto Blue': 'Toronto',
    'San Francisco': 'San Francisco Bay Area',
    'Texas': 'Dallas–Fort Worth',
    'Oakland': 'San Francisco Bay Area',
    'Carolina': 'Charlotte',
    'New England': 'Boston',
    'Tennessee': 'Nashville',
}

nfl_df['City'] = nfl_df['City'].replace(replacement_dict_nfl)

# Eliminar filas que no tienen valores numéricos en las columnas 'W' y 'L'
nfl_df = nfl_df[nfl_df['W'].str.isnumeric() & nfl_df['L'].str.isnumeric()]

# Convertir las columnas 'W' y 'L' a tipo numérico
nfl_df['W'] = pd.to_numeric(nfl_df['W'])
nfl_df['L'] = pd.to_numeric(nfl_df['L'])

nfl_df



,City,Team,W,L,year
1,Boston,New England Patriots,11,5,2018
2,Miami–Fort Lauderdale,Miami Dolphins,7,9,2018
3,Buffalo,Buffalo Bills,6,10,2018
4,New York City,New York Jets,4,12,2018
6,Baltimore,Baltimore Ravens,10,6,2018
7,Pittsburgh,Pittsburgh Steelers,9,6,2018
8,Cleveland,Cleveland Browns,7,8,2018
9,Cincinnati,Cincinnati Bengals,6,10,2018
11,Houston,Houston Texans,11,5,2018
12,Indianapolis,Indianapolis Colts+,10,6,2018


In [319]:
# Seleccionar columnas relevantes de cities
columnas_cities = ["Metropolitan area", "Population (2016 est.)[8]", "NFL"]
cities_subset = cities[columnas_cities].copy()

# Limpiar y procesar datos en cities_subset (ajustar según sea necesario)
cities_subset['NFL'] = cities_subset['NFL'].str.extract('([a-zA-Z\s]+)')
cities_subset = cities_subset.dropna(subset=['NFL'])
cities_subset = cities_subset[~cities_subset['NFL'].str.contains('note')].reset_index(drop=True)
cities_subset = cities_subset.drop(13).reset_index(drop=True)

cities_subset

,Metropolitan area,Population (2016 est.)[8],NFL
0,New York City,20153634,Giants Jets
1,Los Angeles,13310447,Rams Chargers
2,San Francisco Bay Area,6657982,ers Raiders
3,Chicago,9512999,Bears
4,Dallas–Fort Worth,7233323,Cowboys
5,"Washington, D.C.",6131977,Redskins
6,Philadelphia,6070500,Eagles
7,Boston,4794447,Patriots
8,Minneapolis–Saint Paul,3551036,Vikings
9,Denver,2853077,Broncos


In [320]:
# Obtener una serie booleana que indica si cada ciudad en 'Metropolitan Area' está presente en 'City'
matching_cities_series = cities_subset['Metropolitan area'].isin(nfl_df['City'])

# Restablecer el índice de matching_cities_series
matching_cities_series = matching_cities_series.reset_index(drop=True)

# Filtrar las ciudades que coinciden
matching_cities = cities_subset[matching_cities_series]

# Filtrar las ciudades que no coinciden
cities_not_in_nfl_df = cities_subset[~matching_cities_series]

# Mostrar los resultados
print("Ciudades que coinciden en ambas DataFrames:")
print(matching_cities[['Metropolitan area']])

print("\nCiudades en cities_subset que no están en nfl_df:")
print(cities_not_in_nfl_df[['Metropolitan area']])

Ciudades que coinciden en ambas DataFrames:
         Metropolitan area
0            New York City
1              Los Angeles
2   San Francisco Bay Area
3                  Chicago
4        Dallas–Fort Worth
5         Washington, D.C.
6             Philadelphia
7                   Boston
8   Minneapolis–Saint Paul
9                   Denver
10   Miami–Fort Lauderdale
11                 Phoenix
12                 Detroit
13                 Houston
14                 Atlanta
15          Tampa Bay Area
16              Pittsburgh
17               Cleveland
18                 Seattle
19              Cincinnati
20             Kansas City
21               Baltimore
22               Charlotte
23            Indianapolis
24               Nashville
25             New Orleans
26                 Buffalo
27            Jacksonville
28               Green Bay

Ciudades en cities_subset que no están en nfl_df:
Empty DataFrame
Columns: [Metropolitan area]
Index: []


In [321]:
# Fusionar DataFrames
merged_df = pd.merge(nfl_df, cities_subset, left_on='City', right_on='Metropolitan area', how='inner')
merged_df = merged_df.drop(['Metropolitan area', 'NFL', 'year'], axis=1)

# Agrupar por 'City' y 'Team'
grouped_df = merged_df.groupby(['City', 'Team', 'Population (2016 est.)[8]']).agg({
    'W': 'sum',
    'L': 'sum'
}).reset_index()

# Calcular las tasas de victorias y derrotas para cada equipo en cada ciudad
grouped_df['Win_Loss_Ratio'] = grouped_df['W'] / (grouped_df['W'] + grouped_df['L'])

# Agrupar por 'City' y calcular la media de las tasas de victorias y derrotas para cada equipo
merged_df = grouped_df.groupby('City').agg({
    'Win_Loss_Ratio': 'mean',
    'Population (2016 est.)[8]': lambda x: x.iloc[0]
}).reset_index()
merged_df['Population (2016 est.)[8]'] = pd.to_numeric(merged_df['Population (2016 est.)[8]'], errors='coerce')

merged_df

,City,Win_Loss_Ratio,Population (2016 est.)[8]
0,Atlanta,0.437500,5789700
1,Baltimore,0.625000,2798886
2,Boston,0.687500,4794447
3,Buffalo,0.375000,1132804
4,Charlotte,0.437500,2474314
5,Chicago,0.750000,9512999
6,Cincinnati,0.375000,2165139
7,Cleveland,0.466667,2055612
8,Dallas–Fort Worth,0.625000,7233323
9,Denver,0.375000,2853077


In [322]:
def nfl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [323]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

FileNotFoundError: [Errno 2] No such file or directory: 'assets/mlb.csv'